In [1]:
# rsync the data locally
! gcloud storage rsync -r \
    gs://dsgt-clef-erisk-2024/task1/processed/data/count/v3/data \
    /mnt/data/erisk/task1/processed/data/count/v3/data

At gs://dsgt-clef-erisk-2024/task1/processed/data/count/v3/data/**, worker process 377247 thread 139662684215104 listed 501...


In [ ]:
! gcloud storage rsync -r \
    gs://dsgt-clef-erisk-2024/task1/processed/data/word2vec/v3/data \
    /mnt/data/erisk/task1/processed/data/word2vec/v3/data

At gs://dsgt-clef-erisk-2024/task1/processed/data/word2vec/v3/data/**, worker process 374185 thread 140082212865856 listed 501...
At file:///mnt/data/erisk/task1/processed/data/word2vec/v3/data/**, worker process 374185 thread 140082212865856 listed 501...
  Completed files 0 | 0B                                                       


In [ ]:
! gcloud storage rsync -r \
    gs://dsgt-clef-erisk-2024/task1/processed/eval/logistic_word2vec \
    /mnt/data/erisk/task1/processed/eval/logistic_word2vec

At gs://dsgt-clef-erisk-2024/task1/processed/eval/logistic_word2vec/**, worker process 375339 thread 139913874536256 listed 503...
At file:///mnt/data/erisk/task1/processed/eval/logistic_word2vec/**, worker process 375339 thread 139913874536256 listed 503...
  Completed files 0 | 0B                                                       


In [7]:
! gcloud storage cp \
    'gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/g_rels_consenso.csv' \
    '/mnt/data/erisk/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/g_rels_consenso.csv'

Copying gs://dsgt-clef-erisk-2024/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/g_rels_consenso.csv to file:///mnt/data/erisk/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/g_rels_consenso.csv
  Completed files 1/1 | 406.8kiB/406.8kiB                                      


In [8]:
gcs_prefix = "gs://dsgt-clef-erisk-2024"
local_prefix = "/mnt/data/erisk"

In [9]:
from erisk.utils import get_spark

spark = get_spark(
    memory="30g",
    **{
        "spark.sql.parquet.enableVectorizedReader": False,
    },
)
df = spark.read.parquet("/mnt/data/erisk/task1/processed/data/word2vec/v3/data")
df.printSchema()
df.show()
df.count()

root
 |-- docid: string (nullable = true)
 |-- text: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- dataset: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered_words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- word2vec: array (nullable = true)
 |    |-- element: double (containsNull = true)

+------------+--------------------+-----------+-------+--------------------+--------------------+--------------------+
|       docid|                text|   filename|dataset|               words|      filtered_words|            word2vec|
+------------+--------------------+-----------+-------+--------------------+--------------------+--------------------+
| 321176_0_16|Surprisingly, it ...| s_322.trec|   test|[surprisingly,, i...|[surprisingly,, w...|[0.00493731946750...|
|  414896_0_3|Things were good ...| s_415.trec|   test|[things, were, go...|[things, good, wh...|[0.04

19806893

In [12]:
# let's run inference on the entire dataset
from pyspark.ml import PipelineModel
from pyspark.ml.functions import array_to_vector, vector_to_array
from pyspark.sql import functions as F

pipeline = PipelineModel.load(
    f"{local_prefix}/task1/processed/eval/logistic_word2vec/v3/model"
)
pipeline

PipelineModel_759c626141cc

In [13]:
vector_df = df.withColumn("word2vec", array_to_vector("word2vec"))

transformed = pipeline.transform(vector_df)
# transformed.printSchema()

subset = transformed.select(
    "docid",
    "dataset",
    # keep the predictions
    *[c for c in transformed.columns if c.endswith("_prediction")],
    # let's also keep the probability of it being relevant
    *[
        vector_to_array(c).getItem(1).alias(c)
        for c in transformed.columns
        if c.endswith("_probability")
    ],
)
subset.printSchema()
subset.show(vertical=True, n=1)

root
 |-- docid: string (nullable = true)
 |-- dataset: string (nullable = true)
 |-- target_1_prediction: double (nullable = false)
 |-- target_10_prediction: double (nullable = false)
 |-- target_11_prediction: double (nullable = false)
 |-- target_12_prediction: double (nullable = false)
 |-- target_13_prediction: double (nullable = false)
 |-- target_14_prediction: double (nullable = false)
 |-- target_15_prediction: double (nullable = false)
 |-- target_16_prediction: double (nullable = false)
 |-- target_17_prediction: double (nullable = false)
 |-- target_18_prediction: double (nullable = false)
 |-- target_19_prediction: double (nullable = false)
 |-- target_2_prediction: double (nullable = false)
 |-- target_20_prediction: double (nullable = false)
 |-- target_21_prediction: double (nullable = false)
 |-- target_3_prediction: double (nullable = false)
 |-- target_4_prediction: double (nullable = false)
 |-- target_5_prediction: double (nullable = false)
 |-- target_6_predictio

24/04/01 07:31:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0-------------------------------------
 docid                 | 321176_0_16          
 dataset               | test                 
 target_1_prediction   | 0.0                  
 target_10_prediction  | 0.0                  
 target_11_prediction  | 0.0                  
 target_12_prediction  | 0.0                  
 target_13_prediction  | 0.0                  
 target_14_prediction  | 0.0                  
 target_15_prediction  | 0.0                  
 target_16_prediction  | 0.0                  
 target_17_prediction  | 0.0                  
 target_18_prediction  | 0.0                  
 target_19_prediction  | 0.0                  
 target_2_prediction   | 0.0                  
 target_20_prediction  | 0.0                  
 target_21_prediction  | 0.0                  
 target_3_prediction   | 0.0                  
 target_4_prediction   | 0.0                  
 target_5_prediction   | 0.0                  
 target_6_prediction   | 0.0                  
 target_7_pre

24/04/01 07:31:07 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [14]:
# write the results to disk
subset.repartition(32).write.parquet(
    f"{local_prefix}/task1/processed/subset/word2vec_logistic_predictions/v1"
)

In [10]:
subset = spark.read.parquet(
    f"{local_prefix}/task1/processed/subset/word2vec_logistic_predictions/v1"
)

In [13]:
# now let's figure out what percentage of the documtents are actually relevant to some degree
from functools import reduce

relevance = subset.withColumn(
    "relevance_count",
    # sum all the predictions and see if its greater than 0
    reduce(
        lambda a, b: a + b,
        [subset[c] for c in subset.columns if c.endswith("_prediction")],
    ),
)

relevance.groupBy(F.expr("relevance_count > 0")).count().show()
relevance.groupBy("relevance_count").count().orderBy("relevance_count").show()

+---------------------+--------+
|(relevance_count > 0)|   count|
+---------------------+--------+
|                 true|  877548|
|                false|18929345|
+---------------------+--------+



+---------------+--------+
|relevance_count|   count|
+---------------+--------+
|            0.0|18929345|
|            1.0|  474386|
|            2.0|  163794|
|            3.0|   51995|
|            4.0|   86616|
|            5.0|   41301|
|            6.0|   35027|
|            7.0|   14621|
|            8.0|    6082|
|            9.0|    1155|
|           10.0|     296|
|           11.0|    1239|
|           12.0|     187|
|           13.0|     808|
|           14.0|       1|
|           15.0|      30|
|           16.0|       1|
|           17.0|       2|
|           19.0|       1|
|           20.0|       2|
+---------------+--------+
only showing top 20 rows



In [14]:
# let's check how many overlap with the scores we computed before
scored = spark.read.parquet(f"{local_prefix}/task1/processed/data/count_scores/v1")
scored.printSchema()

root
 |-- docid: string (nullable = true)
 |-- score: double (nullable = true)



In [22]:
relevant_scored = relevance.where("relevance_count > 0").join(
    scored.where("score > 6.5"), "docid"
)
relevant_scored.count()

104

In [23]:
df.join(relevant_scored, "docid").select("docid", "score", "text").orderBy(
    F.desc("score")
).show(truncate=200)

+------------+-----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|       docid|            score|                                                                                                                                                                                                    text|
+------------+-----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|s_1721_303_1|8.987321812850125|fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fuck fu...|
|s_1721_301_0|8.502891406705377|fuck fuck fuck fuck fuck fuck fu

In [15]:
# also read the relevant test documents
train_labels_path = f"{local_prefix}/task1/training/t1_training/TRAINING DATA (2023 COLLECTION)/g_rels_consenso.csv"
labels = spark.read.csv(train_labels_path, header=True, inferSchema=True).repartition(
    16
)
labels.printSchema()
labels.show()

root
 |-- query: integer (nullable = true)
 |-- q0: integer (nullable = true)
 |-- docid: string (nullable = true)
 |-- rel: integer (nullable = true)

+-----+---+-------------+---+
|query| q0|        docid|rel|
+-----+---+-------------+---+
|   21|  0| s_2061_384_1|  0|
|    3|  0|  s_1551_51_6|  0|
|    7|  0|s_1674_10_210|  0|
|   16|  0| s_2606_919_3|  0|
|    9|  0| s_614_1573_2|  0|
|   15|  0| s_993_1060_1|  1|
|    9|  0|  s_785_291_1|  0|
|   19|  0| s_617_1608_3|  0|
|   11|  0|  s_835_17_51|  0|
|   13|  0|  s_3045_10_2|  0|
|   14|  0| s_552_399_29|  0|
|   15|  0|s_1404_332_10|  0|
|   15|  0|s_1514_320_53|  0|
|    5|  0| s_2606_507_5|  0|
|   18|  0| s_2597_379_0|  0|
|    6|  0|  s_363_13_29|  0|
|    8|  0|  s_451_465_0|  0|
|    7|  0| s_2986_379_1|  1|
|   21|  0|s_3008_149_15|  0|
|    1|  0| s_2724_252_1|  0|
+-----+---+-------------+---+
only showing top 20 rows



In [16]:
labels.select("docid").distinct().count()

16148

In [23]:
df.join(labels.select("docid").distinct(), "docid").groupBy("dataset").count().show()

+-------+-----+
|dataset|count|
+-------+-----+
|  train|16148|
+-------+-----+



In [26]:
# let's create a new dataset that only contains the most relevant documents for each class
from pyspark.sql.window import Window

primary_key = "docid"
target_probs = [c for c in relevance.columns if c.endswith("_probability")]
top_docs = []
filtered = (
    relevance.where("relevance_count > 0")
    .where("dataset = 'test'")
    .join(scored.where("score < 6.5").select("docid"), "docid")
    .cache()
)
for c in target_probs:
    ordered = filtered.select(
        F.lit(int(c.split("_")[1])).alias("symptom_number"),
        primary_key,
        F.col(c).alias("score"),
    ).where(F.col("score") > 0.7)
    top_docs.append(ordered)

# union all the documents together
k = 10_000
docs = (
    reduce(lambda a, b: a.union(b), top_docs)
    .withColumn(
        "rank",
        F.row_number().over(
            Window.partitionBy("symptom_number").orderBy(F.desc("score"))
        ),
    )
    # keep the top 10_000 documents for each class
    .where(F.col("rank") <= k)
    # now we only care about the documents
    .select(primary_key)
    .distinct()
)

# let's write out the set of documents to disk
docs.repartition(32).write.parquet(
    f"{local_prefix}/task1/processed/data/count_docids_relevant/v1", mode="overwrite"
)

24/04/01 09:14:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [28]:
docs = spark.read.parquet(
    f"{local_prefix}/task1/processed/data/count_docids_relevant/v1"
)
docs.count()

59867

In [29]:
# now that we have a much smaller set of documents to work with, we can embed these documents specifically and run inference on them
(
    df.join(docs, "docid")
    .union(df.join(labels.select("docid").distinct(), "docid"))
    .select("docid", "dataset", "text", "word2vec")
    .repartition(64)
    .write.parquet(
        f"{local_prefix}/task1/processed/data/word2vec_relevant/v1", mode="overwrite"
    )
)

In [30]:
res = spark.read.parquet(f"{local_prefix}/task1/processed/data/word2vec_relevant/v1")
res.groupBy("dataset").count().show()

+-------+-----+
|dataset|count|
+-------+-----+
|  train|16148|
|   test|59867|
+-------+-----+



In [31]:
spark.stop()